# Salary Predictions Based on Job Descriptions

# Part 1 - DEFINE

### ---- 1 Define the problem ----

The assignment I received only came with this instruction:
`Your job as a data scientist is in this assignment is to examine a set of job postings with salaries and then predict salaries for a new set of job postings.`

The data provided in both the training and testing dataset has the following features:
- companyId
- jobType
- degree
- major
- industry
- yearsExperience
- milesFromMetropolis

I didn't receive any additional background or information about why I'd be asked to do this. Understanding the "why" behind any data science project is crucial. If this were a real project, I would ask the following questions:

- Who are the ultimate end users of this project?
- Where does the source data come from?
- What are the main issues they are trying to solve?
- How are they solving those issues right now?
- What are the issue/disadvantages of the current approach?
- What are the key components to a successful model?

**For the purposes of this exercise, I will assume the following:**

The human resources department has requested this project. They obtained the data about job posting salaries from an independent salary research firm. The research firm claims that the data was collected within the last year from comparable companies.

The HR department would like to ensure that the company offers competitive salaries, not too high or too low. They would like to include a predicted salary as a reference point in their decision of whether or not to approve salaries for job postings.

At the moment, they are using rough salary bands provided by the salary reasearch firm as a reference point. However, these are only based on the generic job title (CEO, CFO, Janitor, etc), and the HR department feels that it would be good to include a few other factors to get a more specific salary estimate.

A successful model would be able to predict a salary based on the features provided in the datasets.

**Other items to consider**

In order to make a more accurate model, it would be good to include other features in the data. For example, the specific market that the job is in will likely have a large impact on the salary. Also, there are other components to compensation that may be worth considering as well, such as bonus, vacation days, etc. It would also be good to have more information about things like company size. In my model, I haven't removed any industries, but it would likely be appropriate to focus only on the industry of the target client. 

While job postings are an interesting data point, it is important to remember that they do not represent the actual salaries ultimately agreed upon. It would be good to consider the cost of acquiring actual salary data which could be used in defining guidelines for appropriate salary bands and salary negotiations.

In [1]:
import pandas as pd
#import sklearn as sk

__author__ = "Steve Anderson"
__email__ = "steve@ranksmarts.com"

## Part 2 - DISCOVER

### ---- 2 Load the data ----

In [2]:
train_feat = pd.read_csv('data/train_features.csv',index_col='jobId')
print("Rows and columns in train_feat:",train_feat.shape)
train_feat.head(3)

Rows and columns in train_feat: (1000000, 7)


,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
jobId,,,,,,,
JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83
JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73
JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38


In [3]:
train_salaries = pd.read_csv('data/train_salaries.csv',index_col='jobId')
print("Rows and columns in train_salaries:",train_salaries.shape)
train_salaries.head(3)

Rows and columns in train_salaries: (1000000, 1)


,salary
jobId,
JOB1362684407687,130
JOB1362684407688,101
JOB1362684407689,137


In [4]:
test_feat = pd.read_csv('data/test_features.csv',index_col='jobId')
print("Rows and columns in test_feat:",test_feat.shape)
test_feat.head(3)

Rows and columns in test_feat: (1000000, 7)


,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
jobId,,,,,,,
JOB1362685407687,COMP33,MANAGER,HIGH_SCHOOL,NONE,HEALTH,22,73
JOB1362685407688,COMP13,JUNIOR,NONE,NONE,AUTO,20,47
JOB1362685407689,COMP10,CTO,MASTERS,BIOLOGY,HEALTH,17,9


### ---- 3 Clean the data ----

In [5]:
#Check for duplicates in all feature columns of training data

features = ['companyId', 'jobType', 'degree', 'major', 'industry',
       'yearsExperience', 'milesFromMetropolis']

train_feat_dups = train_feat[train_feat.duplicated(features,keep=False)]
print("Number of rows with duplicate data in train_feat:",len(train_feat_dups))

Number of rows with duplicate data in train_feat: 15917


This shows that there are 15,917 records in the training set that have at least one other record that is identical on all features. I'm curious to see if we include the salaries for those records if they will also be identical, or if they'll be different.

In [6]:
# Add salary column to 'train_feat' data

train_w_salary = train_feat.join(train_salaries)
print("Rows and columns in train_w_salary:",test_feat.shape)
train_w_salary.head(3)

Rows and columns in train_w_salary: (1000000, 7)


,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
jobId,,,,,,,,
JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83,130
JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73,101
JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38,137


In [7]:
features2 = ['companyId', 'jobType', 'degree', 'major', 'industry',
       'yearsExperience', 'milesFromMetropolis', 'salary']

train_w_salary_dups = train_w_salary[train_w_salary.duplicated(features2,keep=False)]
print("Number of rows with duplicate data in train_w_salary:",len(train_w_salary_dups))

Number of rows with duplicate data in train_w_salary: 372


It looks like there are a lot fewer rows (372 vs 15,917) that are duplicates on all features when salary is included. This means that there are many rows that are the same on all characteristics except for salary. Let's take a look at some of these records to see if we can figure out what's going on.

In [8]:
#First 10 records of data that are identical except for jobId and salary

dups_w_salary = train_w_salary[train_feat.duplicated(features,keep=False)]
dups_w_salary.sort_values(features).head(10)

,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
jobId,,,,,,,,
JOB1362685003735,COMP0,CEO,BACHELORS,BIOLOGY,SERVICE,23,34,214
JOB1362685283347,COMP0,CEO,BACHELORS,BIOLOGY,SERVICE,23,34,122
JOB1362685004580,COMP0,CEO,HIGH_SCHOOL,NONE,AUTO,0,82,129
JOB1362685288664,COMP0,CEO,HIGH_SCHOOL,NONE,AUTO,0,82,97
JOB1362685165440,COMP0,CEO,HIGH_SCHOOL,NONE,AUTO,15,13,125
JOB1362685283913,COMP0,CEO,HIGH_SCHOOL,NONE,AUTO,15,13,156
JOB1362684435928,COMP0,CEO,HIGH_SCHOOL,NONE,AUTO,23,94,136
JOB1362684748853,COMP0,CEO,HIGH_SCHOOL,NONE,AUTO,23,94,105
JOB1362684556793,COMP0,CEO,HIGH_SCHOOL,NONE,EDUCATION,11,63,106


This looks strange. First of all, it appears as if most of these have been duplicated, but with VERY different salary data. Just look at the first two rows. A CEO position requiring 23 years of experience, 34 miles from a metropolis, with a salary of 122 or 214? This doesn't make any sense.

In [9]:
# Let me look at some high level numbers to see if anything else looks weird in this dataset

dups_w_salary.describe()

,yearsExperience,milesFromMetropolis,salary
count,15917.000000,15917.000000,15917.000000
mean,11.869825,49.637683,95.311679
std,7.196198,29.097380,37.297904
min,0.000000,0.000000,17.000000
25%,6.000000,24.000000,67.000000
50%,12.000000,49.000000,91.000000
75%,18.000000,75.000000,119.000000
max,24.000000,99.000000,260.000000


In [10]:
# And reviewing the non-numerical fields as well

text_features = ['companyId','jobType','degree','major','industry']
dups_w_salary[text_features].describe()

,companyId,jobType,degree,major,industry
count,15917,15917,15917,15917,15917
unique,63,8,5,9,7
top,COMP32,JANITOR,NONE,NONE,FINANCE
freq,308,6811,7409,14733,2380


It doesn't appear that this anomaly is concentrated in any single feature, although most of the issues do appear when the major is "none". I don't think I'm going to dig any deeper at this point. It just makes sense to remove these duplicat rows. Since I have no way of knowing which salary number is correct, I'll remove the duplicates from all rows.

In [11]:
#drop duplicates from train_feat dataset

train_feat.drop(dups_w_salary.index,inplace=True)
train_feat.shape

(984083, 7)

In [12]:
#drop duplicates from train_salary dataset

train_salaries.drop(dups_w_salary.index,inplace=True)
train_salaries.shape

(984083, 1)

In [13]:
#review duplicates in test_feat dataset

filt = test_feat.duplicated(features,keep=False)
test_feat[filt].sort_values(features)

,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
jobId,,,,,,,
JOB1362685464148,COMP0,CEO,HIGH_SCHOOL,NONE,AUTO,4,80
JOB1362686275559,COMP0,CEO,HIGH_SCHOOL,NONE,AUTO,4,80
JOB1362685526996,COMP0,CEO,HIGH_SCHOOL,NONE,HEALTH,22,83
JOB1362686243166,COMP0,CEO,HIGH_SCHOOL,NONE,HEALTH,22,83
JOB1362685975334,COMP0,CEO,NONE,NONE,EDUCATION,16,61
...,...,...,...,...,...,...,...
JOB1362686221098,COMP9,VICE_PRESIDENT,NONE,NONE,HEALTH,24,23
JOB1362685955412,COMP9,VICE_PRESIDENT,NONE,NONE,SERVICE,5,15
JOB1362686128323,COMP9,VICE_PRESIDENT,NONE,NONE,SERVICE,5,15


In [14]:
#since we will be predicting these salaries, we will drop all non-unique rows

test_feat.drop_duplicates(features,inplace=True)
test_feat.shape

(991954, 7)

In [15]:
train_w_salary.drop(dups_w_salary.index,inplace=True)
train_w_salary.shape

(984083, 8)

In [16]:
#Review numerical columns for other anomalies
train_w_salary.describe()

,yearsExperience,milesFromMetropolis,salary
count,984083.000000,984083.000000,984083.000000
mean,11.994368,49.527506,116.397440
std,7.212639,28.874178,38.649051
min,0.000000,0.000000,0.000000
25%,6.000000,25.000000,89.000000
50%,12.000000,50.000000,114.000000
75%,18.000000,75.000000,141.000000
max,24.000000,99.000000,301.000000


It looks like there are some rows with a salary of zero. Let's take a look at those records.

In [17]:
#Select rows with zero salary

zero_salary = train_w_salary['salary'] <= 0
train_w_salary[zero_salary]

,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
jobId,,,,,,,,
JOB1362684438246,COMP44,JUNIOR,DOCTORAL,MATH,AUTO,11,7,0
JOB1362684903671,COMP34,JUNIOR,NONE,NONE,OIL,1,25,0
JOB1362685059763,COMP25,CTO,HIGH_SCHOOL,NONE,AUTO,6,60,0
JOB1362685223816,COMP42,MANAGER,DOCTORAL,ENGINEERING,FINANCE,18,6,0
JOB1362685235843,COMP40,VICE_PRESIDENT,MASTERS,ENGINEERING,WEB,3,29,0


In [18]:
#Drop rows with zero salary from datasets

train_feat.drop(train_w_salary[zero_salary].index,inplace=True)
train_salaries.drop(train_w_salary[zero_salary].index,inplace=True)
train_w_salary.drop(train_w_salary[zero_salary].index,inplace=True)
train_salaries

,salary
jobId,
JOB1362684407687,130
JOB1362684407688,101
JOB1362684407689,137
JOB1362684407690,142
JOB1362684407691,163
...,...
JOB1362685407682,88
JOB1362685407683,160
JOB1362685407684,64


### ---- 4 Explore the data (EDA) ----

In [19]:
#summarize each feature variable
#summarize the target variable
#look for correlation between each feature and the target
#look for correlation between features

### ---- 5 Establish a baseline ----

In [20]:
#select a reasonable metric (MSE in this case)
#create an extremely simple model and measure its efficacy
#e.g. use "average salary" for each industry as your model and then measure MSE
#during 5-fold cross-validation

### ---- 6 Hypothesize solution ----

In [21]:
#brainstorm 3 models that you think may improve results over the baseline model based
#on your 

Brainstorm 3 models that you think may improve results over the baseline model based on your EDA and explain why they're reasonable solutions here.

Also write down any new features that you think you should try adding to the model based on your EDA, e.g. interaction variables, summary statistics for each group, etc

## Part 3 - DEVELOP

You will cycle through creating features, tuning models, and training/validing models (steps 7-9) until you've reached your efficacy goal

#### Your metric will be MSE and your goal is:
 - <360 for entry-level data science roles
 - <320 for senior data science roles

### ---- 7 Engineer features  ----

In [22]:
#make sure that data is ready for modeling
#create any new features needed to potentially enhance model

### ---- 8 Create models ----

In [23]:
#create and tune the models that you brainstormed during part 2

### ---- 9 Test models ----

In [24]:
#do 5-fold cross validation on models and measure MSE

### ---- 10 Select best model  ----

In [25]:
#select the model with the lowest error as your "prodcuction" model

## Part 4 - DEPLOY

### ---- 11 Automate pipeline ----

In [26]:
#write script that trains model on entire training set, saves model to disk,
#and scores the "test" dataset

### ---- 12 Deploy solution ----

In [27]:
#save your prediction to a csv file or optionally save them as a table in a SQL database
#additionally, you want to save a visualization and summary of your prediction and feature importances
#these visualizations and summaries will be extremely useful to business stakeholders

### ---- 13 Measure efficacy ----

We'll skip this step since we don't have the outcomes for the test data